# Fine-tuning SecBERT LM for text classification
Mount your own drive space as working space with the following three commands



In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cd '/content/drive/MyDrive/secBert'

Upload into the secBert Directory in your drive space dataset's files

In [ ]:
import os
os.chdir('/content/drive/MyDrive/secBert')

In [ ]:
!ls /var/cuda-repo-9-0-local | grep .pub
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!sudo apt-get install cuda-9.0
!nvcc --version
!nvidia-smi

ls: cannot access '/var/cuda-repo-9-0-local': No such file or directory
gpg: can't open '/var/cuda-repo-9-0-local/7fa2af80.pub': No such file or directory
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [787 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease


In [ ]:
!pip install pandas
!pip3 install torch torchvision
!pip install transformers
!pip install sklearn 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.0 MB/s 
     |████████████████████████████████| 596 kB 64.7 MB/s 
     |████████████████████████████████| 6.6 MB 26.0 MB/s 
     |████████████████████████████████| 84 kB 4.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import pandas as pd 

from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer, AutoModelForMaskedLM, BertConfig, AutoModel
from sklearn.preprocessing import LabelEncoder

from time import sleep

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("jackaduma/SecBERT")

pretrained_model = AutoModelForMaskedLM.from_pretrained("jackaduma/SecBERT")
config = BertConfig.from_pretrained("jackaduma/SecBERT", output_hidden_states=True)

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/321M [00:00<?, ?B/s]

Set the Runtime to GPU and check and set cuda availability with the following snippet

In [ ]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

Substitute dataset file name with your own

In [ ]:
df = pd.read_csv('./dataset_mixed_new.csv') #Change name here
df = df.reset_index()

LABELS = len(df['label_tec'].value_counts())

#Encoding labels
encoder = LabelEncoder()
encoder.fit(df['label_tec'])
df['enc_label'] = encoder.transform(df['label_tec'])

df = df[['label_tec', 'sentence', 'enc_label']]
LABELS

188

In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 1e-05


In [ ]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        sentence = str(self.data.sentence[index])
        sentence = " ".join(sentence.split())
        inputs = self.tokenizer.encode_plus(
            sentence,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        if 'enc_label' not in self.data:
            return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long)
            }

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.enc_label[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

Choose an appropriate name for saving your train and test datasets

In [ ]:
from sklearn.model_selection import train_test_split

#Split dataset into train and validation
train_indices, test_indices = train_test_split(list(range(len(df.enc_label))), test_size=0.2, stratify=df.enc_label)
train_dataset = df.copy().drop(test_indices).reset_index(drop=True)
test_dataset = df.copy().drop(train_indices).reset_index(drop=True)

test_dataset.to_csv('test_dataset_mixed_new.csv') #Change name here
train_dataset.to_csv('train_dataset_mixed_new.csv')

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
testing_set = Triage(test_dataset, tokenizer, MAX_LEN)

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0 
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

FULL Dataset: (14412, 3)
TRAIN Dataset: (11529, 3)
TEST Dataset: (2883, 3)


In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class SecBERTClass(torch.nn.Module):
    def __init__(self, pretrained_model_name: str, num_classes: int = None, dropout: float = 0.3):
        super().__init__()
        config = BertConfig.from_pretrained(pretrained_model_name, output_hidden_states=True)
        self.model = AutoModel.from_pretrained(pretrained_model_name, config=config).base_model #pick only the main body of the model
        #for param in self.model.parameters():
          #param.requires_grad = False
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(dropout)
        self.classifier = torch.nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        output_1 = self.model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
#LOAD
model = SecBERTClass("jackaduma/SecBERT", LABELS)

Some weights of the model checkpoint at jackaduma/SecBERT were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.to(device)

In [ ]:
len(train_dataset)

11529

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

# Function to calcuate the accuracy of the model
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [ ]:
torch.cuda.empty_cache()

# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    examples = len(train_dataset)
    losses = [None] * len(training_loader)
    model.train()
    #loop = tqdm(enumerate(training_loader), total=len(training_loader), leave=False)
    for i, data in enumerate(training_loader, 0):
        #print(i)
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)

        losses[i] = loss.item()
        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses):.5f}")
    return 


for epoch in range(EPOCHS):
    train(epoch)

Cost at epoch 0 is 4.24377
Cost at epoch 1 is 2.86740
Cost at epoch 2 is 2.16037
Cost at epoch 3 is 1.70414
Cost at epoch 4 is 1.36501
Cost at epoch 5 is 1.09033
Cost at epoch 6 is 0.86766
Cost at epoch 7 is 0.67973
Cost at epoch 8 is 0.52760
Cost at epoch 9 is 0.41367


In [ ]:
def check_accuracy(loader, model):

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
      for i, data in enumerate(loader, 0):
          x = data['ids'].to(device, dtype = torch.long)
          mask = data['mask'].to(device, dtype = torch.long)
          y = data['targets'].to(device, dtype = torch.long)

          scores = model(x, mask)
          _, predictions = scores.max(1)
          num_correct += (predictions == y).sum()
          num_samples += predictions.size(0)

      print(
          f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
      )



Save the model for further tests

In [ ]:
model_path = '.model_mixed.pt'
#SAVE
torch.save(model.state_dict(), model_path)

In [ ]:
check_accuracy(testing_loader, model)

Got 2026 / 2883 with accuracy 70.27
